# Curso de Optimización (DEMAT)
## Tarea 9

| Descripción:                         | Fechas               |
|--------------------------------------|----------------------|
| Fecha de publicación del documento:  | **Abril 28, 2022**   |
| Fecha límite de entrega de la tarea: | **Mayo   8, 2022**   |


### Indicaciones

- Envie el notebook que contenga los códigos y las pruebas realizadas de cada ejercicio.
- Si se requiren algunos scripts adicionales para poder reproducir las pruebas,
  agreguelos en un ZIP junto con el notebook.
- Genere un PDF del notebook y envielo por separado.

In [122]:
import numpy as np
from scipy.optimize import line_search

EPS = np.finfo(float).eps

## Ejercicio 1. (2 puntos)

Programar las siguientes funciones y sus gradientes, de modo que dependan de la dimensión $n$ de la variable $\mathbf{x}$:


- Función "Tridiagonal 1" generalizada

$$  f(x) = \sum_{i=1}^{n-1} (x_i + x_{i+1} - 3)^2 + (x_i - x_{i+1} + 1)^4  $$


- Función generalizada de Rosenbrock

$$  f(x) = \sum_{i=1}^{n-1} 100(x_{i+1} - x_i^2)^2 + (1 - x_{i} )^2  $$


In [123]:
# Implementación de la funciones y sus gradientes
import numpy as np

def tridiagonal_one(x):
    t = np.copy(x).squeeze()[:-1]
    s = np.copy(x).squeeze()[1:]
    return sum((t+s-3)**2 + (t-s)**4)

def tridiagonal_one_gradient(x):
    t = np.copy(x).squeeze()[:-1]
    s = np.copy(x).squeeze()[1:]
    t = np.concatenate((2*((t+s-3) - 2*(t-s+1)**3),np.array([0])))
    s = np.concatenate((np.array([0]),2*((t+s-3) - 2*(t-s+1)**3)))
    return np.concatenate((2*((t+s-3) - 2*(t-s+1)**3),np.array([0]))) + np.concatenate((np.array([0]),2*((t+s-3) - 2*(t-s+1)**3)))

def rosenbrock(x):
    t = np.copy(x).squeeze()[:-1]
    s = np.copy(x).squeeze()[1:]
    return sum(100*(t-s**2)**2 + (1-t)**2)

def rosenbrock_gradient(x):
    t = np.copy(x).squeeze()[:-1]
    s = np.copy(x).squeeze()[1:]
    t = np.concatenate((2*((t+s-3) - 2*(t-s+1)**3),np.array([0])))
    s = np.concatenate((np.array([0]),2*((t+s-3) - 2*(t-s+1)**3)))
    return np.concatenate((2*((t+s-3) - 2*(t-s+1)**3),np.array([0]))) + np.concatenate((np.array([0]),2*((t+s-3) - 2*(t-s+1)**3)))


```


```
---


## Ejercicio 1 (8 puntos)

Programar y probar el método BFGS modificado.


1. Programar el algoritmo descrito en la diapositiva 16 de la clase 23.
   Agregue una variable $res$ que indique si el algoritmo terminó
   porque se cumplió que la magnitud del gradiente es menor que la toleracia
   dada.

In [124]:
def ModificacionBFGS(x0, tol, funcion, grad, H0, MAX_IT):
    k = 0
    xk = np.copy(x0)
    Hk = np.copy(H0)
    N, N = Hk.shape
    I = np.identity(N)
    res = 0
    
    alpha_f = lambda x, p : line_search(funcion, grad, x, p)

    while k < MAX_IT:
        gk = grad(xk)

        if np.linalg.norm(gk) < tol:
            res = 1
            break
        
        pk = -np.matmul(gk.T,Hk.T)

        if np.matmul(gk.T, pk) > 0:
            lambda_1 = 10**(-5) + (np.matmul(gk.T, pk) / np.matmul(gk.T, gk))[0][0]
            Hk = Hk + lambda_1 * I
            pk = pk - lambda_1 * gk
        
        alpha_k = alpha_f(xk, pk)[0]
        xk_1 = xk + alpha_k * pk
        gk_1 = grad(xk_1)
        sk = xk_1 - xk
        yk = gk_1 - gk

        if np.matmul(yk.T, sk) < 0:
            lambda_2 = 10**(-5) - (np.matmul(yk.T, sk) / np.matmul(sk.T, sk))[0][0]
            Hk_1 = Hk + lambda_2 * I
        else :
            rhok = 1 / (np.matmul(yk.T, sk)[0][0])
            Hk_1 = ( 
                (I - rhok * np.matmul(sk, yk.T)) @ 
                Hk @ 
                (I - rhok * np.matmul(yk, sk.T)) +
                rhok* np.matmul(sk, sk.T) 
            )
        
        k = k + 1
        xk = xk_1
        Hk = Hk_1
        gk = gk_1

    return res, k, xk


2. Fije el número de iteraciones máximas a $N=50000$, 
   y la tolerancia $\tau = \epsilon_m^{1/3}$, donde $\epsilon_m$
   es el épsilon máquina, para terminar las iteraciones 
   si la magnitud del gradiente es menor que $\tau$.
   En cada caso, imprima los siguiente datos:
   
- $n$,
- $f(x_0)$, 
- Usando la variable $res$, imprima un mensaje que indique si
  el algoritmo convergió,
- el  número $k$ de iteraciones realizadas,
- $f(x_k)$,
- la norma del vector $\nabla f_k$, y
- las primeras y últimas 4 entradas del punto $x_k$ que devuelve el algoritmo.

In [125]:
def probarBFGSModificado(funcion, grad, x0, H0):
    N = x0.shape[0]
    res, k, xk = ModificacionBFGS(x0, EPS**(1/3), funcion, grad, H0, 50000)
    mensaje = "El algoritmo convergio" if res == 1 else "El algoritmo no convergio"
    fx0 = funcion(x0)
    norma = np.linalg.norm(grad(xk))

    print(f"""
    N = {N}
    fx0 = {fx0}
    {mensaje}
    k = {k}
    Norma = {norma}
    {xk[0,:4]} ... {xk[0, -4:]}
    """)
    return

3. Probar el algoritmo con las funciones del Ejercicio 1
   con la matriz $H_0$ como la matriz identidad y el 
   punto inicial $x_0$ como:

- La función generalizada de Rosenbrock: 

$$ x_0 = (-1.2, 1, -1.2, 1, ..., -1.2, 1) \in \mathbb{R}^n$$

- La función Tridiagonal 1 generalizada: 

$$ x_0 = (2,2, ..., 2) \in \mathbb{R}^n $$
  
  Pruebe el algoritmo con la dimensión $n=2, 10 , 100$.

In [126]:
for n in [2, 10, 100]:
    I = np.identity(n)
    x0 = np.array([ [-1.2, 1] * (n // 2) ]).T
    probarBFGSModificado(rosenbrock, rosenbrock_grad, x0, I)


    N = 2
    fx0 = 24.199999999999996
    El algoritmo convergio
    k = 34
    Norma = 1.0506606472597589e-07
    [1.00000001] ... [1.00000001]
    

    N = 10
    fx0 = 2057.0
    El algoritmo convergio
    k = 79
    Norma = 3.8065248406907815e-07
    [1.] ... [1.]
    

    N = 100
    fx0 = 24926.000000000022
    El algoritmo convergio
    k = 481
    Norma = 4.451490414011326e-06
    [1.] ... [1.]
    


In [127]:
for n in [2, 10, 20]:
    I = np.identity(n)
    x0 = np.array([ [-1.2, 1] * (n // 2) ]).T
    probarBFGSModificado(tridiagonal, tridiagonal_grad, x0, I)


    N = 2
    fx0 = 12.313600000000003
    El algoritmo convergio
    k = 16
    Norma = 5.696183696343298e-06
    [1.00500746] ... [1.00500746]
    

    N = 10
    fx0 = 521.9584000000001
    El algoritmo convergio
    k = 32
    Norma = 3.0291171643043342e-06
    [1.02464635] ... [1.02464635]
    

    N = 100
    fx0 = 6255.4624000000085
    El algoritmo convergio
    k = 125
    Norma = 1.118182174353982e-06
    [1.02448172] ... [1.02448172]
    
